In [1]:
!pip install -q diffusers

In [2]:
import torch
from huggingface_hub import login
from transformers import pipeline
from diffusers import DiffusionPipeline, FluxPipeline,StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import os
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
hf_token = os.getenv('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\ma2\.cache\huggingface\token
Login successful


In [4]:

def modify_existing_image(image_path, prompt):
    # Load the existing image from a local file
    existing_image = Image.open(image_path).convert("RGB")

    # Load the InstructPix2Pix pipeline
    model_id = "timbrooks/instruct-pix2pix"
    pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
    pipe.to("cpu")

    # Set the scheduler
    pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

    # Generate the modified image
    images = pipe(prompt=prompt, image=existing_image, num_inference_steps=20, image_guidance_scale=1.5).images

    # Save the modified image
    modified_image = images[0]
    modified_image.save("modified_image.png")

    return modified_image


In [5]:
def _modify_existing_image(image_path, prompt):
    # Load the existing image from a local file
    existing_image = Image.open(image_path).convert("RGB")

    # Load the Flux pipeline
    pipe = FluxPipeline.from_pretrained('black-forest-labs/FLUX.1-schnell', torch_dtype=torch.bfloat16).to('cpu')

    # Set the generator
    generator = torch.Generator(device="cpu").manual_seed(0)

    # Generate the modified image using the GPU
    images = pipe(
        prompt=prompt,
        guidance_scale=0.0,
        num_inference_steps=4,
        max_sequence_length=256,
        generator=generator,
        image=existing_image
    ).images

    # Save the modified image
    modified_image = images[0]
    modified_image.save("modified_image.png")

    return modified_image




In [ ]:
# Example usage
image_path = "./images/profilepix.jpg"  # Replace with your local image path
prompt = "A futuristic class full of students learning AI coding in the surreal style of Salvador Dali"
prompt = """
            Generate a new face photo based on the image with a different hairstyle and shirt. 
            Make tha face cleanly shaven in the surreal style of Salvador Dali
        """
modified_image = modify_existing_image(image_path, prompt)
modified_image.show()  # Display the modified image

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\ma2\.conda\envs\generative-llm-projects\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]